In [126]:
import requests
from collections import defaultdict
import pandas as pd
import numpy as np
import datetime

##  APIs

1. Forest: AG.LND.FRST.ZS
2. Permanent cropland: AG.LND.CROP.ZS
3. Agricultural Land: AG.LND.AGRI.ZS
4. Arable Land: AG.LND.ARBL.ZS
5. Land Cereal production : AG.LND.CREL.HA
6. Rural population: SP.RUR.TOTL.ZS
7. Urban Population:SP.URB.TOTL.IN.ZS
8. Male Employment:SL.AGR.EMPL.MA.ZS
9. Female Employement: SL.AGR.EMPL.FE.ZS
10. Fertilizer Use: AG.CON.FERT.ZS
11. Cereal Production: AG.YLD.CREL.KG
12. % of GDP value: NV.AGR.TOTL.ZS
13. Total Population: SP.POP.TOTL
14. Under 5 mortality: SH.DYN.MORT
15. Greenhouse Gases: EN.ATM.GHGT.KT.CE
16. CO2 emissions: EN.ATM.CO2E.KT
17. Poverty : SI.POV.DDAY

In [127]:
country_list = ['World','Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas, The', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'British Virgin Islands', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Cayman Islands', 'Central African Republic', 'Chad', 'Channel Islands', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo, Dem. Rep.', 'Congo, Rep.', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba', 'Curacao', 'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt, Arab Rep.', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia', 'Faroe Islands', 'Fiji', 'Finland', 'France', 'French Polynesia', 'Gabon', 'Gambia, The', 'Georgia', 'Germany', 'Ghana', 'Gibraltar', 'Greece', 'Greenland', 'Grenada', 'Guam', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Honduras', 'Hong Kong SAR, China', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran, Islamic Rep.', 'Iraq', 'Ireland', 'Isle of Man', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya', 'Kiribati', "Korea, Dem. People's Rep.", 'Korea, Rep.', 'Kosovo', 'Kuwait', 'Kyrgyz Republic', 'Lao PDR', 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Macao SAR, China', 'Madagascar', 'Malawi', 'Malaysia', 'Maldives', 'Mali', 'Malta', 'Marshall Islands', 'Mauritania', 'Mauritius', 'Mexico', 'Micronesia, Fed. Sts.', 'Moldova', 'Monaco', 'Mongolia', 'Montenegro', 'Morocco', 'Mozambique', 'Myanmar', 'Namibia', 'Nauru', 'Nepal', 'Netherlands', 'New Caledonia', 'New Zealand', 'Nicaragua', 'Niger', 'Nigeria', 'North Macedonia', 'Northern Mariana Islands', 'Norway', 'Oman', 'Pakistan', 'Palau', 'Panama', 'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Puerto Rico', 'Qatar', 'Romania', 'Russian Federation', 'Rwanda', 'Samoa', 'San Marino', 'Sao Tome and Principe', 'Saudi Arabia', 'Senegal', 'Serbia', 'Seychelles', 'Sierra Leone', 'Singapore', 'Sint Maarten (Dutch part)', 'Slovak Republic', 'Slovenia', 'Solomon Islands', 'Somalia', 'South Africa', 'South Sudan', 'Spain', 'Sri Lanka', 'St. Kitts and Nevis', 'St. Lucia', 'St. Martin (French part)', 'St. Vincent and the Grenadines', 'Sudan', 'Suriname', 'Sweden', 'Switzerland', 'Syrian Arab Republic', 'Tajikistan', 'Tanzania', 'Thailand', 'Timor-Leste', 'Togo', 'Tonga', 'Trinidad and Tobago', 'Tunisia', 'Turkiye', 'Turkmenistan', 'Turks and Caicos Islands', 'Tuvalu', 'Uganda', 'Ukraine', 'United Arab Emirates', 'United Kingdom', 'United States', 'Uruguay', 'Uzbekistan', 'Vanuatu', 'Venezuela, RB', 'Vietnam', 'Virgin Islands (U.S.)', 'West Bank and Gaza', 'Yemen, Rep.', 'Zambia', 'Zimbabwe']

In [128]:
%store largest_10_pop_df
%store countries_population_df

Stored 'largest_10_pop_df' (DataFrame)


UsageError: Unknown variable 'countries_population_df'


In [129]:
def removeFormatColumns(df):
    """
    _summary_

    Args:
        df (DataFrame): _description_

    Returns:
        DataFrame: _description_
    """
    df.drop(columns=['indicator','obs_status','decimal','countryiso3code', 'unit'], inplace=True, axis=1)

    return df

def formatColumns(df):
    """
    _summary_

    Args:
        df (DataFrame): _description_
    Returns:
        df (DateFrame):
    """
    
    df["date"] = pd.to_datetime(df["date"]).dt.year


    #turn country feature into just country name
    for i, country in enumerate(df['country']):
        df.loc[i,'country'] = country['value']

    return df

def justCountries(df):
    """
    _summary_
    Args:
        df (DataFrame): _description_
    Returns:
        DataFrame: _description_
    """
    df=df[df['country'].isin(country_list)]
    return df

In [130]:
# list of worldbank API indicators that will be used to form a database
world_bank_api = ['SP.POP.TOTL', 'AG.LND.TOTL.K2', 'AG.LND.FRST.ZS',
'AG.LND.CROP.ZS','AG.LND.AGRI.ZS','AG.LND.ARBL.ZS','AG.LND.CREL.HA', 
'SP.RUR.TOTL.ZS','SP.URB.TOTL.IN.ZS', 'SL.AGR.EMPL.MA.ZS', 'SL.AGR.EMPL.FE.ZS', 'AG.CON.FERT.ZS','AG.YLD.CREL.KG', 'NV.AGR.TOTL.ZS', 'SH.DYN.MORT', 
'EN.ATM.GHGT.KT.CE', 'EN.ATM.CO2E.KT', 'SI.POV.DDAY']

# loop to create a list of URLs from api indicators
urls = []
for indicator in world_bank_api:
    url = 'http://api.worldbank.org/v2/countries/indicators/' + indicator 
    urls.append(url)
dataframe_list = []

# loop to get request each url and iterate through 18 pages of json data, then turn into a list of dataframes.

for url in urls:
    data = []
    try:  
        for page in range(1,18):
            payload = {'format': 'json', 'per_page': '1000', 'date':'1960:2022', 'page':page}     
            r = requests.get(url, params=payload)
            data+=r.json()[1]
        
        dataframe_list.append(pd.DataFrame(data))

    except:
        print('could not load data', url)

In [131]:
world_bank_columns = ['population', 'total_land_sqkm', 'forest_%','crop_%','agricultural_%','arable_%','cereal_grain_hectare', 'rural_pop_%','urban_pop_%', 'male_employement_ag', 'female_employment_ag', 'fertilizer_consump','cereal_yield_kgPerHectare', 'total_gdp_ag_forestry_fishing', 'mortality_under5', 'Total_Greenhouse_gases', 'CO2_emmission','Poverty_under1_90_per_day']

world_bank_df = None

#format and combine datframes into a single dataframe
for i, df in enumerate(dataframe_list):
  df = removeFormatColumns(df)
  df = formatColumns(df)
  df = justCountries(df)
  
  if world_bank_df is not None:
    world_bank_df.insert(loc=len(world_bank_df.columns),column=world_bank_columns[i], 
    value=df['value'])
  else:
    world_bank_df = pd.DataFrame(df)
    world_bank_df.rename(columns={'value' : world_bank_columns[i]}, inplace=True)

In [132]:
# turning % into total features

world_bank_df['total_urban_pop'] = world_bank_df['urban_pop_%']*world_bank_df['population'] / 100

world_bank_df['total_rural_pop'] = world_bank_df['rural_pop_%']*world_bank_df['population'] / 100

world_bank_df['total_forest_land'] = world_bank_df['forest_%']*world_bank_df['total_land_sqkm'] / 100

world_bank_df['total_ag_land'] = world_bank_df['agricultural_%']*world_bank_df['total_land_sqkm'] / 100

world_bank_df['total_perm_crop_land'] = world_bank_df['crop_%']*world_bank_df['total_land_sqkm'] / 100

world_bank_df['total_arable_land'] = world_bank_df['arable_%']*world_bank_df['total_land_sqkm'] / 100


In [133]:
world_df = world_bank_df[world_bank_df['country']=='World']
country_df = world_bank_df[world_bank_df['country']!='World']

In [134]:
#countries with the top 10 populations
largest_10_population = world_bank_df[world_bank_df['date']=='2021'].nlargest(n=11,columns='population')

largest_10_list = [country for country in largest_10_population['country']]

largest_10_pop_df = world_bank_df[world_bank_df['country'].isin(largest_10_list)]

In [135]:
%store world_bank_df
%store largest_10_pop_df

Stored 'world_bank_df' (DataFrame)
Stored 'largest_10_pop_df' (DataFrame)


In [136]:
world_bank_df

,country,date,population,total_land_sqkm,forest_%,crop_%,agricultural_%,arable_%,cereal_grain_hectare,rural_pop_%,...,mortality_under5,Total_Greenhouse_gases,CO2_emmission,Poverty_under1_90_per_day,total_urban_pop,total_rural_pop,total_forest_land,total_ag_land,total_perm_crop_land,total_arable_land
2976,World,2021,7.836631e+09,1.299493e+08,NaN,NaN,NaN,NaN,NaN,43.415141,...,NaN,NaN,NaN,NaN,4.434346e+09,3.402284e+09,NaN,NaN,NaN,NaN
2977,World,2020,7.763933e+09,1.299493e+08,NaN,NaN,NaN,NaN,NaN,43.835106,...,36.6,NaN,NaN,NaN,4.360605e+09,3.403328e+09,NaN,NaN,NaN,NaN
2978,World,2019,7.683806e+09,1.299493e+08,NaN,NaN,NaN,NaN,NaN,44.277620,...,37.7,4.628762e+07,3.434401e+07,NaN,4.281600e+09,3.402207e+09,NaN,NaN,NaN,NaN
2979,World,2018,7.602716e+09,1.299493e+08,NaN,1.320327,36.903521,10.825170,728365199.0,44.722025,...,38.8,4.613280e+07,3.428935e+07,8.7,4.202628e+09,3.400089e+09,NaN,4.795586e+07,1.715755e+06,1.406723e+07
2980,World,2017,7.519371e+09,1.299561e+08,NaN,1.316979,37.008413,10.831252,734598109.0,45.171279,...,40.0,4.520654e+07,3.351454e+07,9.1,4.122775e+09,3.396596e+09,NaN,4.809468e+07,1.711494e+06,1.407587e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16487,Zimbabwe,1964,4.322854e+06,3.868500e+05,NaN,0.258498,29.055189,5.260437,1253793.0,85.908000,...,134.2,NaN,NaN,NaN,6.091766e+05,3.713677e+06,NaN,1.124000e+05,1.000000e+03,2.035000e+04
16488,Zimbabwe,1963,4.178726e+06,3.868500e+05,NaN,0.258498,28.835466,5.131188,1253451.0,86.422000,...,138.6,NaN,NaN,NaN,5.673874e+05,3.611339e+06,NaN,1.115500e+05,1.000000e+03,1.985000e+04
16489,Zimbabwe,1962,4.039209e+06,3.868500e+05,NaN,0.258498,28.615743,5.001939,1352412.0,86.918000,...,142.9,NaN,NaN,NaN,5.284093e+05,3.510800e+06,NaN,1.107000e+05,1.000000e+03,1.935000e+04
16490,Zimbabwe,1961,3.905038e+06,3.868500e+05,NaN,0.258498,28.396019,4.872690,1377015.0,87.179000,...,147.0,NaN,NaN,NaN,5.006649e+05,3.404373e+06,NaN,1.098500e+05,1.000000e+03,1.885000e+04


In [137]:
world_df = world_bank_df[world_bank_df['country']=='World']
country_df = world_bank_df[world_bank_df['country']!='World']

largest_10_population = country_df[country_df['date']==2021].nlargest(n=10,columns='population')

largest_10_list = [country for country in largest_10_population['country']]

top_10_df=world_bank_df[world_bank_df['country'].isin(largest_10_list)]

world_df = world_bank_df[world_bank_df['country'].isin(['World'])]

top_10_df=top_10_df.groupby(by=['date']).sum().reset_index()

world_df.set_index('date',inplace=True)
top_10_df.set_index('date',inplace=True)

world_df.drop(columns={'country'},inplace=True)

#Create a new dataframe for top 10 populations vs. all other countries
top_10_vs_other = pd.DataFrame(columns=world_bank_columns)

for i in range(1960,2022):
        top_10_vs_other.loc[i]=world_df.loc[i] - top_10_df.loc[i]

top_10_df.insert(0,'country','Top_10')
top_10_vs_other.insert(0,'country','Other')

top_10_vs_other = pd.concat((top_10_vs_other,top_10_df))

top_10_vs_other.reset_index(inplace=True)

C:\Users\ingal\AppData\Local\Temp\ipykernel_10116\2803169993.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  world_df.drop(columns={'country'},inplace=True)


In [139]:
top_10_vs_other

,index,country,population,total_land_sqkm,forest_%,crop_%,agricultural_%,arable_%,cereal_grain_hectare,rural_pop_%,...,mortality_under5,Total_Greenhouse_gases,CO2_emmission,Poverty_under1_90_per_day,total_urban_pop,total_rural_pop,total_forest_land,total_ag_land,total_perm_crop_land,total_arable_land
0,1960.0,Other,1.278128e+09,NaN,NaN,NaN,NaN,NaN,NaN,-621.636972,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1961.0,Other,1.301829e+09,7.798451e+07,NaN,-13.613385,-376.384214,-228.008138,218111441.0,-617.427490,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1962.0,Other,1.326376e+09,7.798449e+07,NaN,-13.833295,-378.014791,-228.732378,221880875.0,-613.146149,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1963.0,Other,1.351421e+09,7.798450e+07,NaN,-13.425313,-380.605672,-231.613165,225327406.0,-608.781734,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1964.0,Other,1.376840e+09,7.798450e+07,NaN,-13.410208,-383.124596,-233.424493,228327318.0,-604.400154,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017,NaN,NaN,3.159133e+09,7.804246e+07,NaN,-35.337748,-447.868109,-249.515661,336894012.0,-364.264721,...,-305.8,1.811848e+07,1.285214e+07,-1.7,1.838361e+09,1.320773e+09,NaN,2.958665e+07,939026.404469,6.733597e+06
2018,NaN,NaN,3.203188e+09,7.803567e+07,NaN,-35.353972,-447.317563,-248.724544,337575404.0,-359.714975,...,-295.0,1.814355e+07,1.285398e+07,-46.3,1.870064e+09,1.333124e+09,NaN,2.945383e+07,942904.934301,6.731862e+06
2019,NaN,NaN,3.247082e+09,7.803567e+07,NaN,NaN,NaN,NaN,NaN,-355.174380,...,-284.5,1.803949e+07,1.275918e+07,NaN,1.902039e+09,1.345043e+09,NaN,NaN,NaN,NaN
2020,NaN,NaN,3.290517e+09,7.803567e+07,NaN,NaN,NaN,NaN,NaN,-350.647894,...,-274.5,NaN,NaN,NaN,1.933910e+09,1.356607e+09,NaN,NaN,NaN,NaN
